In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

In [3]:
# Data Preparation

In [4]:
Fs = 52
# we are going to divide the data into windows of 2.5 secs
time_steps = int(Fs*(2.5)) # 130
overlap_size = int((Fs*(2.5))/2) # 65

In [5]:
def prepare_windows(df, time_steps, overlap_size):

    N_FEATURES = 3

    windows = []
    labels = []
    for i in range(0, len(df) - time_steps, overlap_size):
        x = df['x'].values[i: i + time_steps]
        y = df['y'].values[i: i + time_steps]
        z = df['z'].values[i: i + time_steps]
        
        # Retrieve the most often used label in this segment
        label = stats.mode(df['label'][i: i + time_steps])[0][0]
        windows.append([x, y, z])
        labels.append(label)

    # Bring the segments into a better shape
    windows = np.asarray(windows).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return windows, labels

In [6]:
for i in range(1, 16):
    csv_location = "Processed CSV 2/CSV"+str(i)+"_processed2.csv"
    df = pd.read_csv(csv_location, index_col=[0])
    if(i==1):
        X, Y = prepare_windows(df, time_steps, overlap_size)
    else: 
        x, y = prepare_windows(df, time_steps, overlap_size)
        X = np.concatenate((X, x), axis=0)
        Y = np.concatenate((Y, y), axis=0)

In [7]:
X.shape, Y.shape

((3043, 130, 3), (3043,))

In [8]:
# Splitting dataset into training and testing data

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0, stratify = Y)

In [10]:
X_train.shape, X_test.shape

((2130, 130, 3), (913, 130, 3))

In [11]:
X_train[0].shape, X_test[0].shape

((130, 3), (130, 3))

In [14]:
X_train = X_train.reshape(2130, 130, 3, 1)
X_test = X_test.reshape(913, 130, 3, 1)

In [15]:
X_train.shape, X_test.shape

((2130, 130, 3, 1), (913, 130, 3, 1))

In [16]:
# Model preparation using tensorflow keras library

In [20]:
model = Sequential()
model.add(Conv2D(16, (2, 2), activation = 'relu', input_shape = X_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(7, activation='softmax'))

In [21]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 50, validation_data= (X_test, Y_test), verbose=1)

Train on 2130 samples, validate on 913 samples
Epoch 1/50
2130/2130 [==============================] - 1s 596us/sample - loss: 1.8249 - acc: 0.2099 - val_loss: 1.7289 - val_acc: 0.3198
Epoch 2/50
2130/2130 [==============================] - 1s 492us/sample - loss: 1.6839 - acc: 0.3141 - val_loss: 1.6143 - val_acc: 0.3943
Epoch 3/50
2130/2130 [==============================] - 1s 478us/sample - loss: 1.5812 - acc: 0.3620 - val_loss: 1.5521 - val_acc: 0.3724
Epoch 4/50
2130/2130 [==============================] - 1s 468us/sample - loss: 1.5320 - acc: 0.3808 - val_loss: 1.5010 - val_acc: 0.4042
Epoch 5/50
2130/2130 [==============================] - 1s 476us/sample - loss: 1.4946 - acc: 0.4019 - val_loss: 1.4624 - val_acc: 0.4513
Epoch 6/50
2130/2130 [==============================] - 1s 476us/sample - loss: 1.4091 - acc: 0.4300 - val_loss: 1.4499 - val_acc: 0.4765
Epoch 7/50
2130/2130 [==============================] - 1s 475us/sample - loss: 1.3593 - acc: 0.4746 - val_loss: 1.4083 - val